## MLflow's Model Registry

In [6]:
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

### Interacting with the MLflow tracking server

The `MlflowClient` object allows us to interact with...
- an MLflow Tracking Server that creates and manages experiments and runs.
- an MLflow Registry Server that creates and manages registered models and model versions. 

To instantiate it we need to pass a tracking URI and/or a registry URI

In [9]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

client.search_experiments()

[<Experiment: artifact_location='/home/jellyfish/MLOpsCamp/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1685294200498, experiment_id='1', last_update_time=1685294200498, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1685293597243, experiment_id='0', last_update_time=1685293597243, lifecycle_stage='active', name='Default', tags={}>]

In [10]:
client.create_experiment(name="my-cool-experiment")

'2'

Let's check the latest versions for the experiment with id `1`...

In [11]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [12]:
runs

[<Run: data=<RunData: metrics={'rmse': 6.28608183739523}, params={'learning_rate': '0.38681283879104617',
  'max_depth': '18',
  'min_child_weight': '1.148588628778374',
  'objective': 'reg:linear',
  'reg_alpha': '0.1422529043450288',
  'reg_lambda': '0.3540021019352697',
  'seed': '42'}, tags={'mlflow.log-model.history': '[{"run_id": "5c26c923b2f940528a046a1581d88ab6", '
                              '"artifact_path": "models_mlflow", '
                              '"utc_time_created": "2023-05-29 '
                              '14:15:08.599603", "flavors": {"python_function": '
                              '{"loader_module": "mlflow.xgboost", '
                              '"python_version": "3.9.16", "data": "model.xgb", '
                              '"env": {"conda": "conda.yaml", "virtualenv": '
                              '"python_env.yaml"}}, "xgboost": {"xgb_version": '
                              '"1.7.5", "data": "model.xgb", "model_class": '
                      

In [13]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 5c26c923b2f940528a046a1581d88ab6, rmse: 6.2861
run id: 5bcd8dad03f44305ba288e05762e54c0, rmse: 6.2861
run id: bed184f451a340b49cf93aba9d931d53, rmse: 6.2923
run id: ccbc0b0997944ae4b196b1ea2a180ead, rmse: 6.2934
run id: dd68fce9f63c4118a55234cd88d52d45, rmse: 6.2960


In [17]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse > 6.3 AND metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [18]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 12fb0dac0f7b46f1beb257c1474eeb2f, rmse: 6.3001
run id: b80dc7b25c634367b038020b5a7f7a25, rmse: 6.3001
run id: 45e69d5742c1405cb01207da92884879, rmse: 6.3001
run id: 85b68fa099c24b65af4a262d3ab77b88, rmse: 6.3001
run id: 6a0fbdb563a84cff81166ebbefb9a427, rmse: 6.3160


### Interacting with the Model Registry

In this section We will use the `MlflowClient` instance to:

1. Register a new version for the experiment `nyc-taxi-regressor`
2. Retrieve the latests versions of the model `nyc-taxi-regressor` and check that a new version `4` was created.
3. Transition the version `4` to "Staging" and adding annotations to it.

In [19]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [44]:
run_id = "1058b04d80554dc5adb10d2a7fb67846"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
2023/05/29 13:22:25 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: nyc-taxi-regressor, version 5
Created version '5' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1685384545573, current_stage='None', description=None, last_updated_timestamp=1685384545573, name='nyc-taxi-regressor', run_id='1058b04d80554dc5adb10d2a7fb67846', run_link=None, source='/home/jellyfish/MLOpsCamp/mlops-zoomcamp/02-experiment-tracking/mlruns/1/1058b04d80554dc5adb10d2a7fb67846/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=5>

In [23]:
client.search_registered_models()

[<RegisteredModel: aliases={}, creation_timestamp=1685379545405, description='NYC Taxi Predictor for trip duration.', last_updated_timestamp=1685382529623, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1685379566487, current_stage='Staging', description='', last_updated_timestamp=1685379759229, name='nyc-taxi-regressor', run_id='4a8f2b77cf3b411fa2a023ff812d942a', run_link='', source='/home/jellyfish/MLOpsCamp/mlops-zoomcamp/02-experiment-tracking/mlruns/1/4a8f2b77cf3b411fa2a023ff812d942a/artifacts/model', status='READY', status_message=None, tags={'model': 'gradientboostingregressor'}, user_id=None, version=2>,
  <ModelVersion: aliases=[], creation_timestamp=1685382529623, current_stage='None', description=None, last_updated_timestamp=1685382529623, name='nyc-taxi-regressor', run_id='6a0fbdb563a84cff81166ebbefb9a427', run_link=None, source='/home/jellyfish/MLOpsCamp/mlops-zoomcamp/02-experiment-tracking/mlruns/1/6a0fbdb563a84cff81166ebbefb9a427/artifacts/model', status

In [45]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)
latest_versions

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 3, stage: Staging
version: 4, stage: Production
version: 5, stage: None


In [46]:
model_version = 5
new_stage = "Production"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: aliases=[], creation_timestamp=1685384545573, current_stage='Production', description=None, last_updated_timestamp=1685384565146, name='nyc-taxi-regressor', run_id='1058b04d80554dc5adb10d2a7fb67846', run_link=None, source='/home/jellyfish/MLOpsCamp/mlops-zoomcamp/02-experiment-tracking/mlruns/1/1058b04d80554dc5adb10d2a7fb67846/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=5>

In [47]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1685384545573, current_stage='Production', description='The model version 5 was transitioned to Production on 2023-05-29', last_updated_timestamp=1685384604725, name='nyc-taxi-regressor', run_id='1058b04d80554dc5adb10d2a7fb67846', run_link=None, source='/home/jellyfish/MLOpsCamp/mlops-zoomcamp/02-experiment-tracking/mlruns/1/1058b04d80554dc5adb10d2a7fb67846/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=5>

### Comparing versions and selecting the new "Production" model

In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
2. Download the `DictVectorizer` that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
3. Preprocess the test set using the `DictVectorizer` so we can properly feed the regressors.
4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
5. Based on the results, update the "Production" model version accordingly.


**Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.**

In [36]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [37]:
df = read_dataframe("data/green_tripdata_2021-03.parquet")

In [38]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

/tmp/ipykernel_25274/2852634549.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.download_artifacts`` is deprecated since 2.0. This method will be removed in a future release. Use ``mlflow.artifacts.download_artifacts`` instead.
  client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')


'/home/jellyfish/MLOpsCamp/mlops-zoomcamp/02-experiment-tracking/preprocessor'

In [39]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [40]:
X_test = preprocess(df, dv)

In [41]:
target = "duration"
y_test = df[target].values

In [48]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

2023/05/29 13:23:37 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.3.2, required: mlflow==2.3)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


CPU times: user 6.44 s, sys: 37.7 ms, total: 6.48 s
Wall time: 6.53 s


{'rmse': 6.883765131510798}

In [49]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

2023/05/29 13:25:45 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.3.2, required: mlflow==2.3)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


CPU times: user 79.5 ms, sys: 12.3 ms, total: 91.8 ms
Wall time: 102 ms


{'rmse': 6.659623830022513}

In [50]:
client.transition_model_version_stage(
    name=model_name,
    version=2,
    stage="Production",
    archive_existing_versions=True
)

<ModelVersion: aliases=[], creation_timestamp=1685379566487, current_stage='Production', description='', last_updated_timestamp=1685384849405, name='nyc-taxi-regressor', run_id='4a8f2b77cf3b411fa2a023ff812d942a', run_link='', source='/home/jellyfish/MLOpsCamp/mlops-zoomcamp/02-experiment-tracking/mlruns/1/4a8f2b77cf3b411fa2a023ff812d942a/artifacts/model', status='READY', status_message=None, tags={'model': 'gradientboostingregressor'}, user_id=None, version=2>